In [1]:
import requests
import json
import string
base_url = 'http://localhost:3000/'
API_TOKEN = open("./files/token").readline().strip() #"user:password" encoded in base64
headers = {"Authorization": "Basic %s" %API_TOKEN,
           "Content-type": "application/vnd.api+json",
           "Accept": "application/vnd.api+json"}
file_upload_headers = headers.copy()
file_upload_headers.pop('Content-type');

In [2]:
def print_response(response_text):
    if (r.ok):
        obj = json.loads(response_text)
        print("Returned successfully posted object:\n",obj)
        return obj
    else:
        print("Error: ",r.text)
    return None

In [3]:
def init_json_data(file, title=None):
    json_format = json.load(open(file))
    if (title is not None):
        json_format['data']['attributes']['title'] = title
    json_data = json.dumps(json_format)
    return json_data

In [4]:
#delete previous projects
#for i in range (2,7):
#    url = base_url + "projects/%i.json" %i
#    r = requests.delete(url, headers=headers)
#delete data files
#for i in range(7,13):
#    url = base_url + "data_files/%i.json" %i
#    r = requests.delete(url, headers=headers)

### Post a project

In [5]:
proj_file = "./json/post_max_project.json"
json_data = init_json_data(proj_file, "Project LEDA/DYAD")
url = base_url + "projects"
r = requests.post(url, headers=headers, data=json_data)
proj = print_response(r.text)

Returned successfully posted object:
 {'data': {'id': '10', 'type': 'projects', 'attributes': {'avatar': None, 'title': 'Project LEDA/DYAD', 'description': 'example project for the API', 'web_page': 'http://www.lisym.org', 'wiki_page': 'http://www.h-its.org', 'default_license': 'Other (Open)', 'default_policy': {'access': 'view', 'permissions': [{'resource_type': 'people', 'resource_id': '1', 'access': 'manage'}, {'resource_type': 'projects', 'resource_id': '1', 'access': 'download'}, {'resource_type': 'institutions', 'resource_id': '1', 'access': 'view'}]}}, 'relationships': {'organisms': {'data': [{'id': '548322508', 'type': 'organisms'}]}, 'people': {'data': []}, 'institutions': {'data': []}, 'programmes': {'data': []}, 'investigations': {'data': []}, 'studies': {'data': []}, 'assays': {'data': []}, 'data_files': {'data': []}, 'models': {'data': []}, 'sops': {'data': []}, 'publications': {'data': []}, 'presentations': {'data': []}, 'events': {'data': []}, 'documents': {'data': []}},

### Post an Investigation based on the posted project

In [6]:
inv_file = "./json/post_max_investigation.json"
url = base_url + "investigations"
json_data = init_json_data(inv_file, "API test Investigation")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
inv = print_response(r.text)

Returned successfully posted object:
 {'data': {'id': '8', 'type': 'investigations', 'attributes': {'policy': {'access': 'download', 'permissions': [{'resource_type': 'projects', 'resource_id': '10', 'access': 'manage'}]}, 'title': 'API test Investigation', 'description': 'This is a complex investigation', 'other_creators': 'John Smith'}, 'relationships': {'creators': {'data': [{'id': '1', 'type': 'people'}, {'id': '2', 'type': 'people'}]}, 'submitter': {'data': [{'id': '1', 'type': 'people'}]}, 'people': {'data': [{'id': '1', 'type': 'people'}, {'id': '2', 'type': 'people'}]}, 'projects': {'data': [{'id': '10', 'type': 'projects'}]}, 'studies': {'data': []}, 'assays': {'data': []}, 'data_files': {'data': []}, 'models': {'data': []}, 'sops': {'data': []}, 'publications': {'data': [{'id': '1', 'type': 'publications'}]}, 'documents': {'data': []}}, 'links': {'self': '/investigations/8'}, 'meta': {'created': '2018-04-13T08:58:01.683Z', 'modified': '2018-04-13T08:58:02.091Z', 'api_version'

### Post a Study based on the Investigation 

In [7]:
study_file = "./json/post_max_study.json"
url = base_url + "studies"
json_data = init_json_data(study_file, "API test Study" )
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "INV_ID", inv['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
study = print_response(r.text)

Returned successfully posted object:
 {'data': {'id': '5', 'type': 'studies', 'attributes': {'policy': {'access': 'view', 'permissions': [{'resource_type': 'projects', 'resource_id': '10', 'access': 'download'}]}, 'title': 'API test Study', 'description': 'The Study of many things', 'experimentalists': 'Wet lab people', 'other_creators': 'An unregistered person', 'person_responsible_id': '1'}, 'relationships': {'creators': {'data': [{'id': '1', 'type': 'people'}]}, 'submitter': {'data': [{'id': '1', 'type': 'people'}]}, 'people': {'data': [{'id': '1', 'type': 'people'}]}, 'projects': {'data': [{'id': '10', 'type': 'projects'}]}, 'investigation': {'data': {'id': '8', 'type': 'investigations'}}, 'assays': {'data': []}, 'data_files': {'data': []}, 'models': {'data': []}, 'sops': {'data': []}, 'publications': {'data': [{'id': '1', 'type': 'publications'}]}, 'documents': {'data': []}}, 'links': {'self': '/studies/5'}, 'meta': {'created': '2018-04-13T08:58:05.472Z', 'modified': '2018-04-13T0

### Post an Experimental Assay based on the Study

In [8]:
assay_file = "./json/post_max_assay.json"
url = base_url + "assays"
json_data = init_json_data(assay_file, "API Corrected Experimental assay")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "STUDY_ID", study['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
assay = print_response(r.text)

Returned successfully posted object:
 {'data': {'id': '14', 'type': 'assays', 'attributes': {'policy': {'access': 'download', 'permissions': [{'resource_type': 'projects', 'resource_id': '10', 'access': 'manage'}]}, 'title': 'API Corrected Experimental assay', 'description': 'A Northern Blot Assay', 'other_creators': 'Anonymous creator', 'assay_class': {'title': 'Experimental assay', 'key': 'EXP', 'description': None}, 'assay_type': {'label': None, 'uri': 'http://www.mygrid.org.uk/ontology/JERMOntology#Transcriptomics'}, 'technology_type': {'label': None, 'uri': 'http://www.mygrid.org.uk/ontology/JERMOntology#RNA-Seq'}, 'tags': ['Assay-tag1', 'Assay-tag2', 'Assay-tag3']}, 'relationships': {'creators': {'data': [{'id': '2', 'type': 'people'}]}, 'submitter': {'data': [{'id': '1', 'type': 'people'}]}, 'organisms': {'data': [{'id': '548322508', 'type': 'organisms'}]}, 'people': {'data': [{'id': '1', 'type': 'people'}, {'id': '2', 'type': 'people'}]}, 'projects': {'data': [{'id': '10', 'typ

### Post a Data File linked to the posted Assay

In [9]:
#First POST the meta-data
df_file = "./json/post_max_data_file.json"
url = base_url + "data_files"
json_data = init_json_data(df_file, "API Test Data File")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
r = requests.post(url, headers=headers, data=json_data)

df = print_response(r.text) #only metadata object

#Second, add the file (PUT)
url = df['data']['attributes']['content_blobs'][0]['link']  # content_blobs array size = 1
r = requests.put(url, headers=file_upload_headers, files={"file": open('./files/meltdown.pdf', 'rb')})
print("Response Status:",r)

Returned successfully posted object:
 {'data': {'id': '15', 'type': 'data_files', 'attributes': {'policy': {'access': 'download', 'permissions': [{'resource_type': 'projects', 'resource_id': '10', 'access': 'edit'}]}, 'title': 'API Test Data File', 'description': 'This is the description of the test dT file', 'license': 'CC-BY-4.0', 'latest_version': 1, 'tags': ['tag1', 'tag2'], 'versions': [{'version': 1, 'revision_comments': None, 'url': 'http://localhost:3000/data_files/15?version=1'}], 'version': 1, 'revision_comments': None, 'created_at': '2018-04-13T08:58:11.000Z', 'updated_at': '2018-04-13T08:58:11.000Z', 'content_blobs': [{'original_filename': 'a_pdf_file.pdf', 'url': None, 'md5sum': None, 'sha1sum': None, 'content_type': 'application/pdf', 'link': 'http://localhost:3000/data_files/15/content_blobs/61', 'size': None}], 'other_creators': 'John Smith, Jane Smith'}, 'relationships': {'creators': {'data': [{'id': '1', 'type': 'people'}]}, 'submitter': {'data': [{'id': '1', 'type': 

### Post an SOP linked to an Assay

In [10]:
#First POST the meta-data
sop_file = "./json/post_max_sop.json"
url = base_url + "sops"
json_data = init_json_data(sop_file, "API Test SOP")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
sop = print_response(r.text)
#Second, add the file (PUT)
url = sop['data']['attributes']['content_blobs'][0]['link']  #only one file in an Sop 
r = requests.put(url, headers=file_upload_headers, files={"file": open('./files/meltdown.pdf', 'rb')})
print("Response Status: ",r)

Returned successfully posted object:
 {'data': {'id': '6', 'type': 'sops', 'attributes': {'policy': {'access': 'download', 'permissions': [{'resource_type': 'projects', 'resource_id': '10', 'access': 'edit'}]}, 'title': 'API Test SOP', 'description': 'This is the description', 'license': 'CC-BY-4.0', 'latest_version': 1, 'tags': ['tag1', 'tag2'], 'versions': [{'version': 1, 'revision_comments': None, 'url': 'http://localhost:3000/sops/6?version=1'}], 'version': 1, 'revision_comments': None, 'created_at': '2018-04-13T08:58:18.000Z', 'updated_at': '2018-04-13T08:58:18.000Z', 'content_blobs': [{'original_filename': 'a_pdf_file.pdf', 'url': None, 'md5sum': None, 'sha1sum': None, 'content_type': 'application/pdf', 'link': 'http://localhost:3000/sops/6/content_blobs/62', 'size': None}], 'other_creators': 'John Smith, Jane Smith'}, 'relationships': {'creators': {'data': [{'id': '1', 'type': 'people'}]}, 'submitter': {'data': [{'id': '1', 'type': 'people'}]}, 'people': {'data': [{'id': '1', 't

### Post a Document linked to an Assay

In [11]:
#First POST the meta-data
doc_file = "./json/post_max_document.json"
url = base_url + "documents"
json_data = init_json_data(doc_file, "API Test Document")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
doc = print_response(r.text)
#Second, add the file (PUT)
url = doc['data']['attributes']['content_blobs'][0]['link']  #only one file in an Sop 
r = requests.put(url, headers=file_upload_headers, files={"file": open('./files/meltdown.pdf', 'rb')})
print("Response status: ", r)

Returned successfully posted object:
 {'data': {'id': '6', 'type': 'documents', 'attributes': {'policy': {'access': 'download', 'permissions': [{'resource_type': 'projects', 'resource_id': '10', 'access': 'edit'}]}, 'title': 'API Test Document', 'description': 'This is the description', 'license': 'CC-BY-4.0', 'latest_version': 1, 'tags': ['tag1', 'tag2'], 'versions': [{'version': 1, 'revision_comments': None, 'url': 'http://localhost:3000/documents/6?version=1'}], 'version': 1, 'revision_comments': None, 'created_at': '2018-04-13T08:58:21.000Z', 'updated_at': '2018-04-13T08:58:21.000Z', 'content_blobs': [{'original_filename': 'a_pdf_file.pdf', 'url': None, 'md5sum': None, 'sha1sum': None, 'content_type': 'application/pdf', 'link': 'http://localhost:3000/documents/6/content_blobs/63', 'size': None}], 'other_creators': 'John Smith, Jane Smith'}, 'relationships': {'creators': {'data': [{'id': '2', 'type': 'people'}]}, 'submitter': {'data': [{'id': '1', 'type': 'people'}]}, 'people': {'da

### Post a Model linked to an Assay

In [12]:
#First POST the meta-data
model_file = "./json/post_max_model.json"
url = base_url + "models"
json_data = init_json_data(model_file, "API Test Model")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
mod = print_response(r.text)

#Second, add the file (PUT)
blob_url = lambda bi: mod['data']['attributes']['content_blobs'][bi]['link']
blob_type = lambda bi: str.replace(mod['data']['attributes']['content_blobs'][bi]['content_type'], "application/", "")
filename = lambda bi: "./files/model_"+str(bi+1)+"."+blob_type(bi)

for i in range(len(mod['data']['attributes']['content_blobs'])):
    print("uploading", filename(i), "to", blob_url(i))
    r = requests.put(blob_url(i), headers=file_upload_headers, files={"file": open(filename(i), 'rb')})
    print("Response Status:", r)

Returned successfully posted object:
 {'data': {'id': '11', 'type': 'models', 'attributes': {'policy': {'access': 'download', 'permissions': [{'resource_type': 'projects', 'resource_id': '10', 'access': 'edit'}]}, 'title': 'API Test Model', 'description': 'This is the description', 'license': 'CC-BY-4.0', 'latest_version': 1, 'tags': ['tag1', 'tag2'], 'versions': [{'version': 1, 'revision_comments': None, 'url': 'http://localhost:3000/models/11?version=1'}], 'version': 1, 'revision_comments': None, 'created_at': '2018-04-13T08:58:25.000Z', 'updated_at': '2018-04-13T08:58:25.000Z', 'content_blobs': [{'original_filename': 'docs.pdf', 'url': None, 'md5sum': None, 'sha1sum': None, 'content_type': 'application/pdf', 'link': 'http://localhost:3000/models/11/content_blobs/64', 'size': None}, {'original_filename': 'docs2.pdf', 'url': None, 'md5sum': None, 'sha1sum': None, 'content_type': 'application/pdf', 'link': 'http://localhost:3000/models/11/content_blobs/65', 'size': None}, {'original_fi

### Post a Presentation (linked to a Project)

In [13]:
#First POST the meta-data
pr_file = "./json/post_max_presentation.json"
url = base_url + "presentations"
json_data = init_json_data(pr_file, "API Test Presentation")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
r = requests.post(url, headers=headers, data=json_data)
pr = print_response(r.text)
#Second, add the file (PUT)
url = pr['data']['attributes']['content_blobs'][0]['link']  #only one file in a presentation
r = requests.put(url, headers=file_upload_headers, files={"file": open('./files/presentation.pdf', 'rb')})
print("Response Status: ", r)

Returned successfully posted object:
 {'data': {'id': '6', 'type': 'presentations', 'attributes': {'policy': {'access': 'download', 'permissions': [{'resource_type': 'projects', 'resource_id': '10', 'access': 'edit'}]}, 'title': 'API Test Presentation', 'description': 'This is the description', 'license': 'CC-BY-4.0', 'latest_version': 1, 'tags': ['tag1', 'tag2'], 'versions': [{'version': 1, 'revision_comments': None, 'url': 'http://localhost:3000/presentations/6?version=1'}], 'version': 1, 'revision_comments': None, 'created_at': '2018-04-13T08:58:28.000Z', 'updated_at': '2018-04-13T08:58:28.000Z', 'content_blobs': [{'original_filename': 'a_pdf_file.pdf', 'url': None, 'md5sum': None, 'sha1sum': None, 'content_type': 'application/pdf', 'link': 'http://localhost:3000/presentations/6/content_blobs/67', 'size': None}], 'other_creators': 'John Smith, Jane Smith'}, 'relationships': {'creators': {'data': [{'id': '3', 'type': 'people'}]}, 'submitter': {'data': [{'id': '1', 'type': 'people'}]}

### Post a Modeling Assay with linked Assets 

In [14]:
assay_file2 = "./json/post_max_assay2.json"
url = base_url + "assays"
json_data = init_json_data(assay_file2, "API Test Modeling Assay with linked assets")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "STUDY_ID", study['data']['id'])
json_data = str.replace(json_data, "SOP_ID", sop['data']['id'])
json_data = str.replace(json_data, "MODEL_ID", mod['data']['id'])
json_data = str.replace(json_data, "DOCUMENT_ID", doc['data']['id'])
json_data = str.replace(json_data, "DATAFILE_ID", df['data']['id'])

r = requests.post(url, headers=headers, data=json_data)
assay2 = print_response(r.text)

Returned successfully posted object:
 {'data': {'id': '15', 'type': 'assays', 'attributes': {'policy': {'access': 'download', 'permissions': [{'resource_type': 'projects', 'resource_id': '10', 'access': 'manage'}]}, 'title': 'API Test Modeling Assay with linked assets', 'description': 'modeling analysis', 'other_creators': 'Anonymous creator', 'assay_class': {'title': 'Modelling analysis', 'key': 'MODEL', 'description': None}, 'assay_type': {'label': None, 'uri': 'http://www.mygrid.org.uk/ontology/JERMOntology#Cell_cycle'}, 'technology_type': {'label': None, 'uri': None}, 'tags': ['Assay-tag1', 'Assay-tag2', 'Assay-tag3']}, 'relationships': {'creators': {'data': [{'id': '2', 'type': 'people'}]}, 'submitter': {'data': [{'id': '1', 'type': 'people'}]}, 'organisms': {'data': [{'id': '548322508', 'type': 'organisms'}]}, 'people': {'data': [{'id': '1', 'type': 'people'}, {'id': '2', 'type': 'people'}]}, 'projects': {'data': [{'id': '10', 'type': 'projects'}]}, 'investigation': {'data': {'id

### Post a DataFile with remote content (URL) --> similar for other assets

In [15]:
df_file = "./json/post_remote_data_file.json"
url = base_url + "data_files"
json_data = init_json_data(df_file, "API remote Data File")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
json_data = str.replace(json_data, "ASSAY_ID", assay['data']['id'])
r = requests.post(url, headers=headers, data=json_data)

remote_df = print_response(r.text) 

Returned successfully posted object:
 {'data': {'id': '16', 'type': 'data_files', 'attributes': {'policy': {'access': 'no_access', 'permissions': []}, 'title': 'API remote Data File', 'description': None, 'license': None, 'latest_version': 1, 'tags': None, 'versions': [{'version': 1, 'revision_comments': None, 'url': 'http://localhost:3000/data_files/16?version=1'}], 'version': 1, 'revision_comments': None, 'created_at': '2018-04-13T08:58:41.000Z', 'updated_at': '2018-04-13T08:58:41.000Z', 'content_blobs': [{'original_filename': 'data-example.png', 'url': 'http://seek.virtual-liver.de/data_files/1110/content_blobs/1325/download', 'md5sum': None, 'sha1sum': None, 'content_type': 'image/png', 'link': 'http://localhost:3000/data_files/16/content_blobs/68', 'size': None}], 'other_creators': None}, 'relationships': {'creators': {'data': [{'id': '1', 'type': 'people'}]}, 'submitter': {'data': [{'id': '1', 'type': 'people'}]}, 'people': {'data': [{'id': '1', 'type': 'people'}]}, 'projects': {

### Post a Model with remote content

In [16]:
model_file = "./json/post_remote_model.json"
url = base_url + "models"
json_data = init_json_data(model_file, "API remote model example")
json_data = str.replace(json_data, "PROJECT_ID", proj['data']['id'])
r = requests.post(url, headers=headers, data=json_data)

remote_model = print_response(r.text) 

Returned successfully posted object:
 {'data': {'id': '12', 'type': 'models', 'attributes': {'policy': {'access': 'no_access', 'permissions': []}, 'title': 'API remote model example', 'description': None, 'license': None, 'latest_version': 1, 'tags': None, 'versions': [{'version': 1, 'revision_comments': None, 'url': 'http://localhost:3000/models/12?version=1'}], 'version': 1, 'revision_comments': None, 'created_at': '2018-04-13T08:58:44.000Z', 'updated_at': '2018-04-13T08:58:44.000Z', 'content_blobs': [{'original_filename': 'download', 'url': 'http://seek.virtual-liver.de/data_files/1110/content_blobs/1325/download', 'md5sum': None, 'sha1sum': None, 'content_type': 'image/png', 'link': 'http://localhost:3000/models/12/content_blobs/69', 'size': None}, {'original_filename': '1110.xml', 'url': 'http://seek.virtual-liver.de/data_files/1110.xml', 'md5sum': None, 'sha1sum': None, 'content_type': 'application/xml', 'link': 'http://localhost:3000/models/12/content_blobs/70', 'size': None}], 